# Stats in R2lab

## Scope

The working assumption here is that the interesting data is

* the percentage of usage on a given period start .. end

* the total number of relevant accounts and slices

* also we might wish to pinpoint entries in the db that
  correspond to some operation conditions. For example,
  as of march 2018, we are interested in the disabled accounts
  attached to the r2lab site, because we suspect some people in
  this set have been trying to join but that was never acted upon...

## Changelog

* 2018 March; this a rebuild - see `stats-old.py` - of a previously rather ad hoc script. The present version will be OK for mostly 2017 and later, as we ignore the old data stored in json files.

* 2017 November; at that time - again, see `stats-old.py` - we used 2 different sources of data, presumably because of the migration from the omf/rest API to myplc.

## Prerequisite

* we need to have the complete list of slices, but PLCAPI won't let us access to slices that are deleted - so essentially the ones that have expired;

* so in order to compensate for that, you need to run the script `gather-slices.py` on `r2labapi.inria.fr` and then retrieve the corresponding output here - typically a file named `SLICES-2018-03-23.json`

```
[root@r2labapi ~]# cd r2lab-misc/usage-statistics/
[root@r2labapi usage-statistics]# git pull
Updating 17c7136..d892b36
Fast-forward
 usage-statistics/gather-slices.py    | 73 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 usage-statistics/stats-2018-03.ipynb | 39 +++++++++++++++++++++++++++++++++------
 2 files changed, 106 insertions(+), 6 deletions(-)
 create mode 100755 usage-statistics/gather-slices.py
[root@r2labapi usage-statistics]# ./gather-slices.py
(Over)wrote SLICES-2018-03-23.json
```

Then make sure to retrieve that file locally and define its name here:

```
tparment ~/git/r2lab-misc/usage-statistics (master=) $ rsync -ai $(plr r2labapi):r2lab-misc/usage-statistics/SLICES\* .
>f..t.... SLICES-2018-03-23.json
```

In [1]:
slices_filename = "SLICES-2018-03-23.json"

In [2]:
import json

with open(slices_filename) as feed:
    all_slices = json.loads(feed.read())
    print("OK")

OK


# Convenience

This cell is only here so that any change in the code gets reloaded.

In [3]:
%load_ext autoreload
%autoreload 2

# Proxying / password to the API

You need to know the password of the PLCAPI admin account on `r2labapi.inria.fr`

In [4]:
import getpass
account = "root@r2lab.inria.fr"
password = getpass.getpass(f"Enter password for {account} : ")

Enter password for root@r2lab.inria.fr : ········


In [5]:
# let's check that
auth = {'AuthMethod' : 'password',
        'Username' : account,
        'AuthString' : password}

import xmlrpc.client
url = "https://r2labapi.inria.fr:443/PLCAPI/"

In [6]:
# if everything is OK this should print '1'
proxy = xmlrpc.client.ServerProxy(url)
try:
    print(proxy.AuthCheck(auth))
except Exception as e:
    print(f"OOPS, something wrong with {type(e)} - {e}")

1


*********

## Time formats

In [7]:
from timeutils import parse_date, human_readable

In [8]:
p1 = parse_date("2017-01-01")
p1

input_string 2017-01-01


1483225200.0

In [9]:
p2 = parse_date("2017-01-01", end=True)
p2

input_string 2017-01-01


1483311600.0

In [10]:
human_readable(p1)

'2016-12-31T23:00:UTC'

In [11]:
human_readable(p2)

'2017-01-01T23:00:UTC'

## Enter your period of interest

Second argument to `input_date` is the default answer

In [12]:
from timeutils import input_default

In [13]:
sfrom = input_default("Enter starting day", "2017-01-01")

Enter starting day [2017-01-01] : 


In [14]:
suntil = input_default("Enter ending day", "2017-12-31")

Enter ending day [2017-12-31] : 


In [15]:
ifrom, iuntil = parse_date(sfrom), parse_date(suntil, end=True)
print(f"Period of interest: {human_readable(ifrom)} - {human_readable(iuntil)}")
print(f"Total duration = {(iuntil-ifrom)//(24*3600)} days")

input_string 2017-01-01
input_string 2017-12-31
Period of interest: 2016-12-31T23:00:UTC - 2017-12-31T23:00:UTC
Total duration = 365.0 days


# Fetching user accounts

In [16]:
all_accounts = proxy.GetPersons(auth)

print(f"We have {len(all_accounts)} accounts")

We have 82 accounts


In [17]:
#all_accounts[0]

This will add a `login_base` field to each user account, to identify the attached institution:

In [18]:
all_sites = proxy.GetSites(auth)
site_hash = { site['site_id'] : site for site in all_sites}
for account in all_accounts:
    site_ids = account['site_ids']
    if not site_ids:
        account['login_base'] = '(none)'
    if len(site_ids) == 1:
        account['login_base'] = site_hash[account['site_ids'][0]]['login_base']
    else:
        account['login_base'] = '(' + ",".join(site_hash[site_id]['login_base'] for site_id in site_ids) + ')'

# Classifications - import data from excel

Let's load Walid's paper on the accounts; this is to be able to classify usages and accounts into several categories.

In [19]:
import pandas as pd

Originally this is an excel file `accounts-annotations.xls`. We need an extra library to be able to read `xlsx` files directly:

    pip3 install xlrd

In [20]:
# read excel file
df = pd.read_excel('accounts-annotations.xlsx', encoding='cp1252')

In [21]:
# our main index is the 'Mail' column
df = df.set_index('email')

In [22]:
df.iloc[3:6]

,year,month,day,id,country,family,diana,fit,others,comments
email,,,,,,,,,,
ramonreisfontes@gmail.com,2016,12,16,id=006,Brésil,academia,-,-,yes,NaN
navid.nikaein@eurecom.fr,2016,12,16,id=004,France,academia,-,-,yes,NaN
loic.baron@lip6.fr,2016,12,16,id=003,France,academia,-,yes,-,NaN


### Annotate `all_accounts`

In [23]:
for account in all_accounts:
    if not account['enabled']:
        # print(f"ignoring disabled account {account['email']}")
        continue
    try:
        email = account['email']
        excel_row = df.loc[email]
        # academia or industry
        family = excel_row['family']
        if family in ('academia', 'industry'):
            account['family'] = family
        else:
            print(f"Unknown family for {email} ! (in {account['login_base']})")
        if excel_row['diana'] == 'yes':
            account['scope'] = 'diana'
        elif excel_row['fit'] == 'yes':
            account['scope'] = 'fit'
        elif excel_row['others'] == 'yes':
            account['scope'] = 'others'
        else:
            print(f"Unknown scope for {email} ! (in {account['login_base']})")
    except Exception as e:
        print(f'OOPS with email={email} - {type(e)} - {e}')


If the above cell has not printed any warning, we have all active user accounts tagged with 
* `family` as either `academia` or industry
* `scope` as `diana`, `fit` or `others`

# Stats on user accounts

In [24]:
def show_accounts(accounts):
    accounts.sort(key = lambda person: person['date_created'])
    for i, account in enumerate(accounts, 1):
        default = 'n/a' if account['enabled'] else '--'
        print(f"[{i:02d}] "
              f" {'OK' if account['enabled'] else 'KO'}"
              f" {human_readable(account['date_created'])}"
              f" {account['login_base']:22s}",
              f" {account.get('family', default):8s}",
              f" {account.get('scope', default):8s}",
              f" {account['email']}")

In [25]:
# in case you need to see all accounts
# show_accounts(all_accounts)
print(f"we have a total of {len(all_accounts)} accounts")

we have a total of 82 accounts


### narrowing on the selected period

In [26]:
selected_accounts = [
    account for account in all_accounts 
    if account['date_created'] >= ifrom and
       account['date_created'] <= iuntil
]

In [27]:
# show_accounts(selected_accounts)
print(f"a total of {len(selected_accounts)} accounts were created over the selected period")

a total of 58 accounts were created over the selected period


### focusing on enabled accounts (What really matters)

In [28]:
enabled_accounts_in_selected_period = [
    account for account in selected_accounts if account['enabled']
]

In [29]:
print(f"New enabled accounts in the selected period = {len(enabled_accounts_in_selected_period)}")

New enabled accounts in the selected period = 32


In [30]:
show_accounts(enabled_accounts_in_selected_period)

[01]  OK 2017-01-11T11:37:UTC inria                   academia  fit       fabrice.valois@insa-lyon.fr
[02]  OK 2017-01-11T11:37:UTC inria                   academia  fit       jad.oueis@insa-lyon.fr
[03]  OK 2017-01-11T11:37:UTC inria                   academia  fit       razvan.stanica@inria.fr
[04]  OK 2017-01-24T16:22:UTC inria                   academia  diana     arnaud.legout@inria.fr
[05]  OK 2017-02-15T18:19:UTC inria                   academia  diana     mondi.ravi@inria.fr
[06]  OK 2017-02-15T18:25:UTC inria                   academia  diana     mondi.ravi@gmail.com
[07]  OK 2017-03-04T06:46:UTC inria                   academia  others    kishoregolla9@gmail.com
[08]  OK 2017-03-09T09:57:UTC inria                   academia  diana     yassir.mr1@gmail.com
[09]  OK 2017-03-09T19:18:UTC inria                   academia  others    sebastian.fehlandt@inria.cl
[10]  OK 2017-04-06T19:36:UTC inria                   academia  others    victor.codocedo@inria.cl
[11]  OK 2017-04-11T16:

### By scope : diana / fit / others

In [31]:
for scope in ['diana', 'fit', 'others']:
    scope_accounts = [account for account in enabled_accounts_in_selected_period
                       if account['scope'] == scope]
    print(f"in scope {scope}, {len(scope_accounts)} new enabled accounts")

in scope diana, 9 new enabled accounts
in scope fit, 3 new enabled accounts
in scope others, 20 new enabled accounts


### By family : academia / industry

In [32]:
for family in ['academia', 'industry']:
    family_accounts = [account for account in enabled_accounts_in_selected_period
                       if account['family'] == family]
    print(f"in family {family}, {len(family_accounts)} new enabled accounts")

in family academia, 29 new enabled accounts
in family industry, 3 new enabled accounts


# Classifying slices

In [33]:
# sort in expiration order    
all_slices.sort(key = lambda slice: slice['expires'])
    
print(f"Found {len(all_slices)} slices")    

Found 50 slices


In [34]:
def show_slices(slices):
    for i, slice in enumerate(slices, 1):
        print(f"{i:02d} "
            f" created {human_readable(slice['created'])}"
            f" expires {human_readable(slice['expires'])}"
            f" {slice['name']}"
         )

In [35]:
show_slices(all_slices)

01  created 2016-12-16T13:27:UTC expires 2016-12-30T13:27:UTC upmc_mobicom.mobicom_demo
02  created 2016-12-16T13:27:UTC expires 2016-12-30T13:27:UTC upmc_pltutorial.emulation
03  created 2016-12-16T13:27:UTC expires 2016-12-30T13:27:UTC inria_farzaneh.routing
04  created 2016-12-16T13:27:UTC expires 2016-12-30T13:27:UTC unicamp_wifisdn.slicesdn
05  created 2016-12-16T13:27:UTC expires 2016-12-30T13:27:UTC upmc_ieee.infocom2016
06  created 2016-12-16T13:27:UTC expires 2016-12-30T13:27:UTC inria_wifi.sdn
07  created 2016-12-16T13:27:UTC expires 2016-12-30T13:27:UTC inria_iotlab.iotlab_slice
08  created 2016-12-20T15:12:UTC expires 2017-01-03T15:12:UTC inria_mesh.routing
09  created 2017-01-11T11:43:UTC expires 2017-01-25T11:43:UTC inria_citi4g
10  created 2016-12-16T13:27:UTC expires 2017-02-17T14:30:UTC inria_oai.b210
11  created 2016-12-16T13:27:UTC expires 2017-03-05T07:45:UTC inria_oai.skype
12  created 2016-12-16T13:27:UTC expires 2017-03-05T19:27:UTC inria_anas.ping
13  created 20

### Ignoring admin slices

In [36]:
admin_slices = ['auto_', 'nightly', 'maintenance', 'tutorial']

In [37]:
def relevant(slice_or_lease):
    return not any(admin in slice_or_lease['name'] for admin in admin_slices)

### classifying slices

In [38]:
# hash accounts per person_id
accounts_hash = { account['person_id'] : account for account in all_accounts}

for slice in all_slices:
    if not relevant(slice):
        continue
    person_ids = slice['person_ids']
    slice['foo'] = [accounts_hash[person_id].get('family', '???') for person_id in slice['person_ids']]

In [39]:
for slice in all_slices:
    if not relevant(slice):
        continue
    print(slice['name'], slice['foo'])

upmc_mobicom.mobicom_demo []
inria_farzaneh.routing []
unicamp_wifisdn.slicesdn []
upmc_ieee.infocom2016 []
inria_wifi.sdn []
inria_iotlab.iotlab_slice []
inria_mesh.routing []
inria_citi4g []
inria_oai.b210 []
inria_oai.skype []
inria_anas.ping []
inria_andhraiot []
inria_es []
inria_fehland3 []
inria_walid.demo []
inria_codocedo []
inria_mario.script []
inria_fehland1 []
inria_fehland2 []
inria_visit []
eurecoms3_test []
eurecoms3_test2 []
inria_oai.build []
inria_manipooja []
eurecoms3_coexist []
inria_yassir []
inria_urauf []
inria_kazi.puc []
eurecoms3_today []
inria_jawad []
inria_oai.slicing []
inria_cefore ['academia', 'academia', 'academia', 'academia']
inria_batman ['academia']
inria_tp ['academia']
inria_oai ['academia', 'academia', 'academia', 'academia']
inria_r2lab.admin ['academia', '???', 'academia', 'academia']
inria_radiomap ['academia', 'academia', 'academia', 'academia', 'academia']
inria_l2bm ['academia', 'academia', 'academia', 'academia']
inria_pfe ['academia', '

Long story short, at this point we can safely consider that 100% of our usage is academy.

# Fetching leases

In [40]:
# fetch leases for that period
selected_leases = proxy.GetLeases(
    auth,
    {'>t_from' : ifrom, '<t_from' : iuntil}
)
print(f"there have been {len(selected_leases)} reservations made during the period")

there have been 616 reservations made during the period


### Sorting in ascending order

In [41]:
selected_leases.sort(key=lambda lease: lease['t_from'])

### A glimpse

In [42]:
def lease_line(lease):
    return f"{lease['name']:25s} {human_readable(lease['t_from'])} -> {human_readable(lease['t_until'])}"

In [43]:
def glimpse(leases, size=5):
    for lease in leases[:size]:
        print(lease_line(lease))
    print("...")
    for lease in leases[-size:]:
        print(lease_line(lease))

In [44]:
glimpse(selected_leases)

inria_r2lab.nightly       2017-01-06T15:00:UTC -> 2017-01-06T16:00:UTC
inria_naoufal.mesh        2017-01-09T15:30:UTC -> 2017-01-09T16:00:UTC
inria_r2lab.nightly       2017-01-09T16:00:UTC -> 2017-01-09T17:00:UTC
inria_r2lab.nightly       2017-01-11T02:00:UTC -> 2017-01-11T03:00:UTC
inria_r2lab.nightly       2017-01-12T02:00:UTC -> 2017-01-12T03:00:UTC
...
inria_r2lab.nightly       2017-12-27T02:00:UTC -> 2017-12-27T03:00:UTC
inria_pfe                 2017-12-27T14:40:UTC -> 2017-12-27T17:00:UTC
inria_pfe                 2017-12-27T23:00:UTC -> 2017-12-28T03:00:UTC
inria_cefore              2017-12-28T07:10:UTC -> 2017-12-28T08:00:UTC
inria_r2lab.nightly       2017-12-31T02:00:UTC -> 2017-12-31T03:00:UTC


# Usage ratio

##### raw ratio *vs* opening hours

The raw ratio is obtained by comparing the amount of time reserved with the total amount of time available.

Assuming that opening hours would be mon-fri from 09:00 to 19:00

In [45]:
open_correction = (5 * 10) / (7 * 24)
print(f"Opening hours are {open_correction:.2%} of total hours")

Opening hours are 29.76% of total hours


##### user *vs* admin

We try to classify the various slices in 2 families whether they are for management/operations purposes, or used for actual experimentation.

In [46]:
total_duration = iuntil - ifrom

In [47]:
def show_usage_ratio(leases, total_duration, message):

    def duration(lease):
        return lease['t_until'] - lease['t_from']

    reserved_duration = sum(duration(lease) for lease in leases)
    print(f"Total time reserved: {reserved_duration} / {total_duration} s")
    print(f"                i.e: {reserved_duration/3600:.2f} / {total_duration/3600:.2f}    hours")
    print(f"                i.e: {reserved_duration/(24*3600):.2f} / {total_duration/(24*3600):.2f}       days")
    
    raw_ratio = reserved_duration / total_duration
    print(f"{message}: raw_ratio is {raw_ratio:.2%}")
    
    open_ratio = raw_ratio / open_correction
    print(f"{message}: open_ratio is {open_ratio:.2%}")

In [48]:
show_usage_ratio(selected_leases, total_duration, "ALL LEASES")

Total time reserved: 4041600 / 31536000.0 s
                i.e: 1122.67 / 8760.00    hours
                i.e: 46.78 / 365.00       days
ALL LEASES: raw_ratio is 12.82%
ALL LEASES: open_ratio is 43.06%


### Usage ratio - filtered

We discard slices whose name contains any of the following

In [49]:
filtered_leases = [lease for lease in selected_leases if relevant(lease)]

In [50]:
show_usage_ratio(filtered_leases, total_duration, "FILTERED LEASES")

Total time reserved: 3663000 / 31536000.0 s
                i.e: 1017.50 / 8760.00    hours
                i.e: 42.40 / 365.00       days
FILTERED LEASES: raw_ratio is 11.62%
FILTERED LEASES: open_ratio is 39.03%


### validated *vs* non-validated accounts

Still on the selected period, show the ones that were enabled or not

In [51]:
enabled_selected_accounts = [ account for account in selected_accounts if account['enabled']]
disabled_selected_accounts = [ account for account in selected_accounts if not account['enabled']]

In [52]:
show_accounts(enabled_selected_accounts)

[01]  OK 2017-01-11T11:37:UTC inria                   academia  fit       fabrice.valois@insa-lyon.fr
[02]  OK 2017-01-11T11:37:UTC inria                   academia  fit       jad.oueis@insa-lyon.fr
[03]  OK 2017-01-11T11:37:UTC inria                   academia  fit       razvan.stanica@inria.fr
[04]  OK 2017-01-24T16:22:UTC inria                   academia  diana     arnaud.legout@inria.fr
[05]  OK 2017-02-15T18:19:UTC inria                   academia  diana     mondi.ravi@inria.fr
[06]  OK 2017-02-15T18:25:UTC inria                   academia  diana     mondi.ravi@gmail.com
[07]  OK 2017-03-04T06:46:UTC inria                   academia  others    kishoregolla9@gmail.com
[08]  OK 2017-03-09T09:57:UTC inria                   academia  diana     yassir.mr1@gmail.com
[09]  OK 2017-03-09T19:18:UTC inria                   academia  others    sebastian.fehlandt@inria.cl
[10]  OK 2017-04-06T19:36:UTC inria                   academia  others    victor.codocedo@inria.cl
[11]  OK 2017-04-11T16:

In [53]:
show_accounts(disabled_selected_accounts)

[01]  KO 2017-01-10T11:52:UTC inria                   --        --        mario.zancanaro@inria.fr
[02]  KO 2017-02-06T08:48:UTC r2lab                   --        --        raj85.vishwakarma@gmail.com
[03]  KO 2017-03-09T09:12:UTC r2lab                   --        --        yassir.mr4@gmail.com
[04]  KO 2017-05-05T12:27:UTC eurecomple              --        --        kharade.rohan@iitb.ac.in
[05]  KO 2017-05-11T09:48:UTC eurecomple              --        --        info@try-rx.com
[06]  KO 2017-05-19T20:18:UTC eurecomple              --        --        yoseph.maloche@studenti.unitn.it
[07]  KO 2017-05-24T08:09:UTC inria                   --        --        qwerty1643@skku.edu
[08]  KO 2017-06-27T09:29:UTC unicamp                 --        --        christophe.letoquin@orange.com
[09]  KO 2017-07-03T06:52:UTC inria                   --        --        manipooja1990@gmail.com
[10]  KO 2017-08-26T17:09:UTC r2lab                   --        --        rayansalhab@hotmail.com
[11]  KO 2017

*****
*****
*****

# Specific to the stats done in March 2018

## the ones who registered with the `r2lab` site

**NOT** restricted to the selected period

In [54]:
r2lab_selected_accounts = [ account for account in all_accounts if 'r2lab' in account['login_base']]

In [55]:
show_accounts(r2lab_selected_accounts)

[01]  KO 2017-02-06T08:48:UTC r2lab                   --        --        raj85.vishwakarma@gmail.com
[02]  KO 2017-03-09T09:12:UTC r2lab                   --        --        yassir.mr4@gmail.com
[03]  KO 2017-08-26T17:09:UTC r2lab                   --        --        rayansalhab@hotmail.com
[04]  KO 2017-09-04T11:53:UTC r2lab                   --        --        29776716@qq.com
[05]  KO 2017-09-27T18:42:UTC r2lab                   --        --        yuchia.tseng@irt-systemx.fr
[06]  KO 2017-10-01T00:36:UTC r2lab                   --        --        sample@email.tst
[07]  KO 2017-10-03T03:37:UTC r2lab                   --        --        a5566740293@gmail.com
[08]  KO 2017-10-17T21:09:UTC r2lab                   --        --        rayclaytonrudman@hotmail.com
[09]  KO 2017-10-28T13:32:UTC r2lab                   --        --        hishamalzain21@gmail.com
[10]  KO 2017-10-29T09:29:UTC r2lab                   --        --        hisham_alzain21@yahoo.com
[11]  KO 2017-11-04T09:1

## other disabled accounts

In [56]:
other_disabled_accounts = [ account for account in all_accounts 
                           if not account['enabled'] and 'r2lab' not in account['login_base']]

In [57]:
show_accounts(other_disabled_accounts)

[01]  KO 2016-12-16T13:27:UTC inria                   --        --        mario-antonio.zancanaro@inria.fr
[02]  KO 2017-01-10T11:52:UTC inria                   --        --        mario.zancanaro@inria.fr
[03]  KO 2017-05-05T12:27:UTC eurecomple              --        --        kharade.rohan@iitb.ac.in
[04]  KO 2017-05-11T09:48:UTC eurecomple              --        --        info@try-rx.com
[05]  KO 2017-05-19T20:18:UTC eurecomple              --        --        yoseph.maloche@studenti.unitn.it
[06]  KO 2017-05-24T08:09:UTC inria                   --        --        qwerty1643@skku.edu
[07]  KO 2017-06-27T09:29:UTC unicamp                 --        --        christophe.letoquin@orange.com
[08]  KO 2017-07-03T06:52:UTC inria                   --        --        manipooja1990@gmail.com
[09]  KO 2017-08-27T07:31:UTC inria                   --        --        xo79@qq.com
[10]  KO 2017-09-12T14:57:UTC inria                   --        --        zgouaksvic@mailstome.today
[11]  KO 2017-